## **RAG based summarization agent**

LLM based summarization agent that summarizes an automotive issue
using a given dataset

Steps:


1.   Load and Preprocess the data
2.   Create Embeddings and VectorStore for Document Retrieval
3.   Define LLM and Summarization Chain



0. Install requirements


In [ ]:
!pip install transformers -U
!pip install chromadb langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 78.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.3 MB/s eta 0:

In [ ]:
!pip install tokenizers==0.21

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 57.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.5.23 requires tokenizers<=0.20.3,>=0.13.2, but you have tokenizers 0.21.0 which is incompatible.


## 1.   Load and Preprocess the data


In [ ]:
import pandas as pd
import chromadb
from  langchain_groq import ChatGroq
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from sentence_transformers import SentenceTransformer


In [ ]:
#load document for RAG
file_path = '/content/drive/MyDrive/colab/predii/FLAT_RCL.txt'

df = pd.read_csv(
    file_path,
    sep='\t',
    dtype=str ,
    header=None,
    on_bad_lines='skip'
)

df.head()

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,1,02V288000,FORD,FOCUS,2000,02S41,ELECTRICAL SYSTEM:12V/24V/48V BATTERY:CABLES,FORD MOTOR COMPANY,19990719,20010531,...,NaN,NaN,CERTAIN PASSENGER VEHICLES EQUIPPED WITH ZETEC...,"THIS, IN TURN, COULD CAUSE THE BATTERY CABLES ...",DEALERS WILL INSPECT THE BATTERY CABLES FOR TH...,ALSO CONTACT THE NATIONAL HIGHWAY TRAFFIC SAFE...,000015339000215021000000202,NaN,NaN,NaN
1,2,02V288000,FORD,FOCUS,2001,02S41,ELECTRICAL SYSTEM:12V/24V/48V BATTERY:CABLES,FORD MOTOR COMPANY,19990719,20010531,...,NaN,NaN,CERTAIN PASSENGER VEHICLES EQUIPPED WITH ZETEC...,"THIS, IN TURN, COULD CAUSE THE BATTERY CABLES ...",DEALERS WILL INSPECT THE BATTERY CABLES FOR TH...,ALSO CONTACT THE NATIONAL HIGHWAY TRAFFIC SAFE...,000015339000215022000000202,NaN,NaN,NaN
2,3,02V236000,JAYCO,FT EAGLE 10 SG,2003,NaN,EQUIPMENT:OTHER:LABELS,"JAYCO, INC.",20020730,20020813,...,NaN,NaN,"ON CERTAIN FOLDING TENT CAMPERS, THE FEDERAL C...","IF THE TIRES WERE INFLATED TO 80 PSI, THEY COU...",OWNERS WILL BE MAILED CORRECT LABELS FOR INSTA...,"ALSO, CUSTOMERS CAN CONTACT THE NATIONAL HIGHW...",000015210000106403000000349,NaN,NaN,NaN
3,4,02V237000,HOLIDAY RAMBLER,ENDEAVOR,2000,NaN,STRUCTURE,MONACO COACH CORP.,NaN,NaN,...,NaN,NaN,"ON CERTAIN CLASS A MOTOR HOMES, THE FLOOR TRUS...",CONDITIONS CAN RESULT IN THE BOTTOMING OUT THE...,DEALERS WILL INSPECT THE FLOOR TRUSS NETWORK S...,CUSTOMERS CAN ALSO CONTACT THE NATIONAL HIGHWA...,000015211000083965000000272,NaN,NaN,NaN
4,5,02V237000,HOLIDAY RAMBLER,ENDEAVOR,1999,NaN,STRUCTURE,MONACO COACH CORP.,NaN,NaN,...,NaN,NaN,"ON CERTAIN CLASS A MOTOR HOMES, THE FLOOR TRUS...",CONDITIONS CAN RESULT IN THE BOTTOMING OUT THE...,DEALERS WILL INSPECT THE FLOOR TRUSS NETWORK S...,CUSTOMERS CAN ALSO CONTACT THE NATIONAL HIGHWA...,000015211000080938000000272,NaN,NaN,NaN


In [ ]:
# Check size of dataframe
df.shape

(291375, 27)

In [ ]:
# add columns names from metadata
column_names = [
    "RECORD_ID", "CAMPNO", "MAKETXT", "MODELTXT", "YEARTXT", "MFGCAMPNO",
    "COMPNAME", "MFGNAME", "BGMAN", "ENDMAN", "RCLTYPECD", "POTAFF", "ODATE",
    "INFLUENCED_BY", "MFGTXT", "RCDATE", "DATEA", "RPNO", "FMVSS", "DESC_DEFECT",
    "CONEQUENCE_DEFECT", "CORRECTIVE_ACTION", "NOTES", "RCL_CMPT_ID",
    "MFR_COMP_NAME", "MFR_COMP_DESC", "MFR_COMP_PTNO"
]

df.columns = column_names



In [ ]:
# Standardize by converting to uppercase and stripping whitespaces
df['MAKETXT'] = df['MAKETXT'].str.upper().str.strip()

# filtering out only FORT and TOYOTA
filtered_df = df[df['MAKETXT'].isin(['FORD', 'TOYOTA'])].copy()

# verify the unique values
print("Unique MAKETXT values : ", filtered_df['MAKETXT'].unique())



Unique MAKETXT values :  ['FORD' 'TOYOTA']


In [ ]:
filtered_df.head()

,RECORD_ID,CAMPNO,MAKETXT,MODELTXT,YEARTXT,MFGCAMPNO,COMPNAME,MFGNAME,BGMAN,ENDMAN,...,RPNO,FMVSS,DESC_DEFECT,CONEQUENCE_DEFECT,CORRECTIVE_ACTION,NOTES,RCL_CMPT_ID,MFR_COMP_NAME,MFR_COMP_DESC,MFR_COMP_PTNO
0,1,02V288000,FORD,FOCUS,2000,02S41,ELECTRICAL SYSTEM:12V/24V/48V BATTERY:CABLES,FORD MOTOR COMPANY,19990719,20010531,...,NaN,NaN,CERTAIN PASSENGER VEHICLES EQUIPPED WITH ZETEC...,"THIS, IN TURN, COULD CAUSE THE BATTERY CABLES ...",DEALERS WILL INSPECT THE BATTERY CABLES FOR TH...,ALSO CONTACT THE NATIONAL HIGHWAY TRAFFIC SAFE...,000015339000215021000000202,NaN,NaN,NaN
1,2,02V288000,FORD,FOCUS,2001,02S41,ELECTRICAL SYSTEM:12V/24V/48V BATTERY:CABLES,FORD MOTOR COMPANY,19990719,20010531,...,NaN,NaN,CERTAIN PASSENGER VEHICLES EQUIPPED WITH ZETEC...,"THIS, IN TURN, COULD CAUSE THE BATTERY CABLES ...",DEALERS WILL INSPECT THE BATTERY CABLES FOR TH...,ALSO CONTACT THE NATIONAL HIGHWAY TRAFFIC SAFE...,000015339000215022000000202,NaN,NaN,NaN
148,149,02V249000,FORD,CROWN VICTORIA,2002,02S39,"FUEL SYSTEM, OTHER:STORAGE:TANK ASSEMBLY",FORD MOTOR COMPANY,20010510,20020322,...,NaN,NaN,"ON CERTAIN NATURAL GAS MODEL VEHICLES, A T-FIT...",A GAS LEAK OF SUFFICIENT QUANTITY CONCENTRATED...,"DEALERS WILL INSTALL A REDESIGNED ""T"" FITTING ...",ALSO CONTACT THE NATIONAL HIGHWAY TRAFFIC SAFE...,000015251000215009000000162,NaN,NaN,NaN
300,301,02V239000,FORD,NAVIGATOR,2000,02L10,EQUIPMENT:OTHER:LABELS,FORD MOTOR COMPANY,19990331,20000806,...,571,120,CERTAIN 4X2 SPORT UTILITY VEHICLES FAIL TO COM...,CUSTOMERS MAY INFLATE THEIR REAR TIRES BASED O...,A SUPPLEMENTARY LABEL WILL BE SENT TO CUSTOMER...,ALSO CONTACT THE NATIONAL HIGHWAY TRAFFIC SAFE...,000015220000089746000000349,NaN,NaN,NaN
301,302,02V239000,FORD,EXPEDITION,2000,02L10,EQUIPMENT:OTHER:LABELS,FORD MOTOR COMPANY,19990331,20000806,...,571,120,CERTAIN 4X2 SPORT UTILITY VEHICLES FAIL TO COM...,CUSTOMERS MAY INFLATE THEIR REAR TIRES BASED O...,A SUPPLEMENTARY LABEL WILL BE SENT TO CUSTOMER...,ALSO CONTACT THE NATIONAL HIGHWAY TRAFFIC SAFE...,000015220000203219000000349,NaN,NaN,NaN


In [ ]:
#new df with only required 6 columns
df_combined = filtered_df[["MAKETXT", "MODELTXT", "YEARTXT",
                           "DESC_DEFECT", "CONEQUENCE_DEFECT", "CORRECTIVE_ACTION"]].copy()

#combined all issues into single column
df_combined['combined_summary'] = (
    df_combined['DESC_DEFECT'].fillna('') + ' ' +
    df_combined['CONEQUENCE_DEFECT'].fillna('') + ' '  +
    df_combined['CORRECTIVE_ACTION'].fillna('')
)

#dropping reduntant cols
df_combined = df_combined.drop(columns = ["DESC_DEFECT", "CONEQUENCE_DEFECT", "CORRECTIVE_ACTION"])


In [ ]:
df_combined.head()

,MAKETXT,MODELTXT,YEARTXT,combined_summary
0,FORD,FOCUS,2000,CERTAIN PASSENGER VEHICLES EQUIPPED WITH ZETEC...
1,FORD,FOCUS,2001,CERTAIN PASSENGER VEHICLES EQUIPPED WITH ZETEC...
148,FORD,CROWN VICTORIA,2002,"ON CERTAIN NATURAL GAS MODEL VEHICLES, A T-FIT..."
300,FORD,NAVIGATOR,2000,CERTAIN 4X2 SPORT UTILITY VEHICLES FAIL TO COM...
301,FORD,EXPEDITION,2000,CERTAIN 4X2 SPORT UTILITY VEHICLES FAIL TO COM...


In [ ]:
df_combined.shape

(17137, 4)

In [ ]:
#df_combined = df_combined[:150]

## 2.   Create Embeddings and VectorStore for Document Retrieval


In [ ]:
#Using opensource ChromaDB Vectorstore
client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection('auto_issues')

#embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

#collection of documents from combined summary and metadata
if not collection.count():
    for idx, row in df_combined.iterrows():
        embedding = embedding_model.encode(row['combined_summary'])
        collection.add(
            documents=[row['combined_summary']],
            metadatas={
                'make': row['MAKETXT'],
                'model': row['MODELTXT'],
                'year': row['YEARTXT']
            },
            ids=[row['RECORD_ID']],
            embeddings=[embedding]
        )


## 3.   Define LLM and Summarization Chain

```
# This is formatted as code
```



In [ ]:

# From GroqAPI import llama-3.1-8b model
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    groq_api_key=""
)

retrieval_prompt = PromptTemplate(
    input_variables=["issue", "make", "model", "year", "retrieved_documents"],

    template=
    """

    Analyze and summarize information about {issue} for {make} {model} {year}.

    - You should focus on issues specifically matching the vehicle model and year if available.
    - Only stick to the information retrieved from the documents.
    - Summarize into 50 words.

    Retrieved Documents:
    {retrieved_documents}

    Give a concise summary focusing on directly relevant issues
    Do not use any information outside of the retrieved documents.

    """
)

# LLM Chain
retrieval_chain = retrieval_prompt | llm


In [ ]:
# Summarization function
def summarize_issue(input_data):

    make = input_data['make'].upper()
    model = input_data['model'].upper()
    year = input_data['year']
    issue = input_data['issue']

    query_text = f"{make} {model} {year} {issue}"

    #compute query embedding
    query_embedding = embedding_model.encode(query_text)

    #Retrieve relevant documents
    results = collection.query(
        query_embeddings=[query_embedding],
        where={
            "$and": [
                {"make": make},
                {"year": year}]},
        n_results=10 )

    retrieved_docs = []
    seen_docs = set()

    #combine documents and metadata
    for doc, metadata in zip(results.get('documents', [[]])[0],results.get('metadatas', [[]])[0]):
        if doc not in seen_docs:
            retrieved_docs.append( doc)
            seen_docs.add(doc)

    #top 5 unique documents
    final_docs = retrieved_docs[:5]

    inputs = {
        "issue": issue ,
        "make": make,
        "model": model,
        "year": year,
        "retrieved_documents": "\n \n".join(final_docs)}

    # summarization chain
    summary = retrieval_chain.invoke(inputs)
    return {
        'input' : input_data ,
        'retrieved_documents' : final_docs,
        'summary' : summary.content
    }


In [ ]:
output = summarize_issue(input_data)
print(output)

{'input': {'make': 'ford', 'model': 'escape', 'year': '2001', 'issue': 'stuck throttle risk'}, 'retrieved_documents': ['VEHICLE DESCRIPTION:  PASSENGER VEHICLES EQUIPPED WITH ADJUSTABLE PEDALS.    IF THE GREASE FROM THE ADJUSTABLE PEDAL ASSEMBLY ENTERS THE STOP LAMP SWITCH, IT CAN CONTAMINATE THE CONTACTS LEADING TO CARBON BUILD UP, AND POTENTIALLY, A SHORT CIRCUIT. A SHORT CIRCUIT COULD LEAD TO EITHER THE BRAKE LAMPS STAYING ON, OR TO A LOSS OF BRAKE LAMP FUNCTION, INCREASING THE RISK OF A CRASH. DEALERS WILL REPLACE THE BRAKE LAMP SWITCH AND WIPE DOWN THE ADJUSTABLE PEDAL ASSEMBLY TO REMOVE EXCESS GREASE.  OWNER NOTIFICATION BEGAN MARCH 22, 2001.  OWNERS WHO TAKE THEIR VEHICLES TO AN AUTHORIZED DEALER ON AN AGREED UPON SERVICE DATE AND DO NOT RECEIVE THE FREE REMEDY WITHIN A REASONABLE TIME SHOULD CONTACT FORD AT 1-800-392-3673.', 'CERTAIN PASSENGER VEHICLES EQUIPPED WITH ADJUSTABLE PADELS ARE BEING RECALLED IN ORDER TO ADJUST THE BRAKE AND ACCELERATOR PEDALS TO A MINIMUM OF 50 MM OF